## **Import Libraries**

In [1]:
import numpy as np
import pandas as pd

## **Check first file from FHIR**

In [71]:
# filename_test ="/home/cleysonl/fsdl-project/Clinical-Notes-and-DNA/fhir/Alane21_Von197_d839e2fa-a8e0-10e1-38a2-776a8db8ef2c.jso
# filename_test ='/home/cleysonl/fsdl-project/Clinical-Notes-and-DNA/fhir/Aletha771_Waelchi213_7edbfb43-e906-2e27-f6e6-b1a4d096b9b0.json'
# filename_test = '/home/cleysonl/fsdl-project/Clinical-Notes-and-DNA/fhir/Bob965_Streich926_8b645ef6-8ddd-ff98-4a3a-1949c0834829.json'
# filename_test = '/home/cleysonl/fsdl-project/Clinical-Notes-and-DNA/fhir/Fidel864_Spencer878_e136851f-4c79-c867-9545-dc36c9436c60.json'
# filename_test = '/home/cleysonl/fsdl-project/Clinical-Notes-and-DNA/fhir/Glinda951_Anderson154_30503e79-d7ca-7e9f-caba-2059cd3f7619.json'
# filename_test='/home/cleysonl/fsdl-project/Clinical-Notes-and-DNA/fhir/Ryan260_Quitzon246_41ce1a88-6529-3f5f-1576-1f1d7a8623c0.json'
# filename_test = '/home/cleysonl/fsdl-project/Clinical-Notes-and-DNA/fhir/Norbert530_O_Reilly797_c93a9775-f000-9532-0d4d-cf317051d0c4.json'
# filename_test='/home/cleysonl/fsdl-project/Clinical-Notes-and-DNA/fhir/Alda400_Kris249_c04befa5-e835-3f4c-2b90-88da0af27c0d.json'
# filename_test='/home/cleysonl/fsdl-project/Clinical-Notes-and-DNA/fhir/Anthony633_Harber290_0ca7d23f-ac16-3f7a-e3da-f4cc29307867.json'
# filename_test='/home/cleysonl/fsdl-project/Clinical-Notes-and-DNA/fhir/Abe604_Frami345_b8dd1798-beef-094d-1be4-f90ee0e6b7d5.json'
# filename_test = '/home/cleysonl/fsdl-project/Clinical-Notes-and-DNA/fhir/Adriana394_Nader710_34dead0b-b339-74d5-91b4-8211c4bbfc47.json'
filename_test = '/home/cleysonl/fsdl-project/Clinical-Notes-and-DNA/fhir/Kala987_Rowe323_1f0dc7da-34b4-e1d1-afdd-ea7e998522d7.json'
# filename_test='/home/cleysonl/fsdl-project/Clinical-Notes-and-DNA/fhir/Adam631_Turner526_6f60d183-2b8d-8c3c-77d0-2b684653651e.json'
# filename_test='/home/cleysonl/fsdl-project/Clinical-Notes-and-DNA/fhir/Adele475_Kuhn96_2bc08ad5-c34c-ec2e-5777-d7b1ddceeee1.json'


In [72]:
import base64

class DataPatient():
    def read_data(self, filename):
        return pd.read_json(filename)

    def name_id(self, data):
        self.patient_info = data["resource"] 
        self.name_patient = self.patient_info["name"][0]['given'][0] + '_' + self.patient_info["name"][0]['family'] 
        self.patient_id = self.patient_info["id"]
        return [self.name_patient, self.patient_id]

    def encounter_id(self, data):
        if data["resource"]["resourceType"]=='Encounter': 
            return data['fullUrl']

    def condition(self, data):   
            return [data['resource']['encounter']['reference'], data["resource"]["code"]['coding'][0]['display']]
    
    
    def diagnostic_report(self, data):
        if  'presentedForm' in list(data['resource'].keys()) and 'data' in list(data['resource']['presentedForm'][0].keys()):
            # Clean notes and separate the text in complaints, history_illness, social_history
            # allergies and assessment plan and save them in a dictionary
            text= str(base64.b64decode(data['resource']['presentedForm'][0]['data']))[2:-1]
            date = text[2:12]
            chief_complaints = [i.replace('\\n','') for i in text.split('#')[1].split('\\n-')[1:] if i!=""]
            history_illness = [i.replace('\\n','') for i in text.split('#')[2].split('\\n-')[0].split('\\n')[1:] if i!=""]
            social_history = [i.replace('\\n','') for i in text.split('#')[3].split('\\n-')[0].split('\\n')[1:] if i!=""]
            allergies = [i.replace('\\n','') for i in text.split('#')[4].split('\\n-')[0].split('\\n')[1:] if i!=""]
            medication = [i.replace('\\n','') for i in text.split('#')[5].split('\\n-')[0].split('\\n')[1:] if i!=""]
            assessment_plan = [i.replace('\\n','') for i in text.split('#')[6].split('\\n-')[0].split('\\n')[1:] if i!=""]
            plan = [i.replace('\\n','') for i in text.split('#')[7].split('\\n-')[0].split('\\n')[1:] if i!=""]
            doc_ordered ={'date': date, 'chief_complaints':chief_complaints, 'history_illness':history_illness,
                            'social_history': social_history, 'allergies':allergies, 'medication': medication, 
                            'assessment_plan':assessment_plan, 'plan':plan}
            return ['report', doc_ordered, data['resource']['encounter']['reference']]
        elif data["resource"]["code"]["coding"][0]['display'].lower()=='genetic analysis summary panel':
                return ['dna', {}, data['resource']['encounter']['reference']]
        else:
            return None


    def imagingstudy(self, data):              
            return data['resource']['identifier'][0]['value'][8:]
                

In [95]:
def create_df_patient(filename):
    dataframe_patient = pd.DataFrame(columns=['date', 'patient_name', 'patient_id', 'encounter_id', 'Condition',
                                            'ImagingStudy', 'DiagnosticReport', 'DnaStudy'])

    patient = DataPatient()
    data_patient =patient.read_data(filename)
    for j in data_patient['entry'].values:
        if j['resource']['resourceType'] =='Encounter':
            dataframe_patient.loc[dataframe_patient.shape[0],'encounter_id'] = patient.encounter_id(j)

    for i in range(dataframe_patient.shape[0]):
        dataframe_patient.loc[i,'patient_name'], dataframe_patient.loc[i,'patient_id'] = \
                patient.name_id(data_patient.loc[0,'entry'])
    
    for j in data_patient['entry'].values:
        if j['resource']['resourceType'] =='Condition':
            encounter_id, condition=patient.condition(j)
            dataframe_patient.loc[dataframe_patient['encounter_id']==encounter_id,'Condition'] = condition
            
    for j in data_patient['entry'].values:
        if j['resource']['resourceType'] =='DiagnosticReport':
            if patient.diagnostic_report(j) != None:
                doc_type, dict_note, encounter_id = patient.diagnostic_report(j)
                if doc_type == 'dna':
                    dataframe_patient.loc[dataframe_patient['encounter_id']==encounter_id,'DnaStudy'] = \
                        dataframe_patient.loc[dataframe_patient['encounter_id']==encounter_id,'patient_name'] +'_'+ \
                        dataframe_patient.loc[dataframe_patient['encounter_id']==encounter_id,'patient_id'] + "_dna.csv"
                if doc_type == 'report':
                    dataframe_patient.loc[dataframe_patient['encounter_id']==encounter_id,'DiagnosticReport'] = [dict_note]
            
    for j in data_patient['entry'].values:
        if j['resource']['resourceType'] =='ImagingStudy' and j["resource"]["series"][0]['modality']['code']=='MR':
            dataframe_patient.loc[dataframe_patient['encounter_id']==encounter_id,'ImagingStudy'] = \
                dataframe_patient.loc[dataframe_patient['encounter_id']==encounter_id,'patient_name'] +'_'+ \
                dataframe_patient.loc[dataframe_patient['encounter_id']==encounter_id,'patient_id'] + patient.imagingstudy(j)    

    for i in range(dataframe_patient.shape[0]):
        dataframe_patient.loc[i,'date'] = dataframe_patient.loc[i,'DiagnosticReport']['date'] 
            
    # return dataframe_patient[dataframe_patient['Condition'].notna()]
    return dataframe_patient

In [96]:
create_df_patient(filename_test)

,date,patient_name,patient_id,encounter_id,Condition,ImagingStudy,DiagnosticReport,DnaStudy
0,1948-02-10,Kala987_Rowe323,1f0dc7da-34b4-e1d1-afdd-ea7e998522d7,urn:uuid:ef4a387b-8d71-db15-a46e-fe68fef7bd05,Hypertension,NaN,"{'date': '1948-02-10', 'chief_complaints': [],...",NaN
1,1948-07-01,Kala987_Rowe323,1f0dc7da-34b4-e1d1-afdd-ea7e998522d7,urn:uuid:aababa01-c84b-f958-a474-feac625129b0,NaN,NaN,"{'date': '1948-07-01', 'chief_complaints': [],...",NaN
2,1949-02-15,Kala987_Rowe323,1f0dc7da-34b4-e1d1-afdd-ea7e998522d7,urn:uuid:4bea8712-d11a-18f2-f0a3-bb3ccfaeab1a,NaN,NaN,"{'date': '1949-02-15', 'chief_complaints': [],...",NaN
3,1950-02-21,Kala987_Rowe323,1f0dc7da-34b4-e1d1-afdd-ea7e998522d7,urn:uuid:aad1cff5-0757-a849-20ff-ebb1621274de,NaN,NaN,"{'date': '1950-02-21', 'chief_complaints': [],...",NaN
4,1951-02-27,Kala987_Rowe323,1f0dc7da-34b4-e1d1-afdd-ea7e998522d7,urn:uuid:07074648-a7b7-234a-d1bd-2b22a8231e53,NaN,NaN,"{'date': '1951-02-27', 'chief_complaints': [],...",NaN
...,...,...,...,...,...,...,...,...
127,2019-07-25,Kala987_Rowe323,1f0dc7da-34b4-e1d1-afdd-ea7e998522d7,urn:uuid:f42ecd32-a658-9086-01f6-6275fcc80105,NaN,NaN,"{'date': '2019-07-25', 'chief_complaints': [],...",NaN
128,2020-03-24,Kala987_Rowe323,1f0dc7da-34b4-e1d1-afdd-ea7e998522d7,urn:uuid:9a375c30-b9c7-0826-f8e0-e6c2a7866e55,NaN,NaN,"{'date': '2020-03-24', 'chief_complaints': [],...",NaN
129,2020-04-22,Kala987_Rowe323,1f0dc7da-34b4-e1d1-afdd-ea7e998522d7,urn:uuid:c070a983-a270-e4fd-8994-4abff2fe875b,Sprain of ankle,NaN,"{'date': '2020-04-22', 'chief_complaints': [],...",NaN
130,2020-07-24,Kala987_Rowe323,1f0dc7da-34b4-e1d1-afdd-ea7e998522d7,urn:uuid:b74181c1-0fb3-7be7-3526-3d217790c9bb,NaN,NaN,"{'date': '2020-07-24', 'chief_complaints': [],...",NaN


In [97]:
import os
def aggregate_dataframes(function_):
    total_dataframe = pd.DataFrame(columns=['date', 'patient_name', 'patient_id', 'encounter_id', 'Condition',
                                            'ImagingStudy', 'DiagnosticReport', 'DnaStudy'])
    directory = "/home/cleysonl/fsdl-project/Clinical-Notes-and-DNA/fhir/"
    for filename in os.listdir(directory):
        f=os.path.join(directory, filename)
        total_dataframe = pd.concat([total_dataframe,function_(f)], axis=0,ignore_index=True)
    return total_dataframe.reset_index()

In [98]:
total = aggregate_dataframes(create_df_patient)

In [107]:
# total = total.drop('index', axis=1)

total.loc[(total['DnaStudy'].notna()) & (total['Condition'].notna()),:].shape[0]
total.loc[(total['DnaStudy'].notna()),:].head(40)

# total.head()

,index,date,patient_name,patient_id,encounter_id,Condition,ImagingStudy,DiagnosticReport,DnaStudy
480,480,2017-08-29,Kala987_Rowe323,1f0dc7da-34b4-e1d1-afdd-ea7e998522d7,urn:uuid:34fa8111-6b75-0c90-ecb5-ef460632474b,Stroke,NaN,"{'date': '2017-08-29', 'chief_complaints': [],...",Kala987_Rowe323_1f0dc7da-34b4-e1d1-afdd-ea7e99...
503,503,2000-08-04,Inez706_Cassin499,d15fc66c-ae01-1f28-bb51-976b1be31ffc,urn:uuid:7f50a1b7-d8fc-1d2b-4e26-e27d565fb959,Stroke,NaN,"{'date': '2000-08-04', 'chief_complaints': [],...",Inez706_Cassin499_d15fc66c-ae01-1f28-bb51-976b...
532,532,2011-12-12,Val761_Satterfield305,69ac903b-809e-043a-c7b0-9825cd13acae,urn:uuid:292d06ca-5d52-d540-07b3-d96278ceb8a9,Stroke,NaN,"{'date': '2011-12-12', 'chief_complaints': [],...",Val761_Satterfield305_69ac903b-809e-043a-c7b0-...
604,604,1941-11-18,Glinda951_Anderson154,30503e79-d7ca-7e9f-caba-2059cd3f7619,urn:uuid:58069aac-09ed-3c72-a04d-230eedcabb42,Stroke,NaN,"{'date': '1941-11-18', 'chief_complaints': [],...",Glinda951_Anderson154_30503e79-d7ca-7e9f-caba-...
734,734,1953-03-17,Andy346_Hilpert278,a57e3099-1ff2-4292-8fe4-030786381163,urn:uuid:5b889868-763b-2ee2-818f-6210b3559143,Stroke,NaN,"{'date': '1953-03-17', 'chief_complaints': [],...",Andy346_Hilpert278_a57e3099-1ff2-4292-8fe4-030...
945,945,1983-05-28,Donovan745_Wolff180,1b03ceba-207d-21bd-444c-fb8e7eaa3d1d,urn:uuid:1da06066-a2d2-9c36-b504-ffef35186f45,Stroke,NaN,"{'date': '1983-05-28', 'chief_complaints': [],...",Donovan745_Wolff180_1b03ceba-207d-21bd-444c-fb...
977,977,2011-10-29,Chong355_Douglas31,64a578fc-bc82-8ea3-12e6-6569b0820eb7,urn:uuid:dba41d37-f4cf-da23-5741-ea169ca4e6ce,Acute bronchitis (disorder),NaN,"{'date': '2011-10-29', 'chief_complaints': [' ...",Chong355_Douglas31_64a578fc-bc82-8ea3-12e6-656...
1077,1077,2016-05-24,Ricky354_McLaughlin530,f1610c48-cf3c-5383-6ef6-64fedd856808,urn:uuid:d07190e9-89ba-afae-cc9e-75946805fa7b,Stroke,NaN,"{'date': '2016-05-24', 'chief_complaints': [],...",Ricky354_McLaughlin530_f1610c48-cf3c-5383-6ef6...
1123,1123,1972-04-30,Alan320_West559,813a9f44-c25c-df6a-5a62-c9a3e76ad1fc,urn:uuid:8d85e43d-4a56-7946-46ee-934f22480759,History of myocardial infarction (situation),NaN,"{'date': '1972-04-30', 'chief_complaints': [],...",Alan320_West559_813a9f44-c25c-df6a-5a62-c9a3e7...
1603,1603,1954-05-07,Waylon572_Feil794,4a23b7e8-7136-d3d8-3285-6dc792840aeb,urn:uuid:902b58ff-09df-6c03-5af9-95bbfed24a50,Stroke,NaN,"{'date': '1954-05-07', 'chief_complaints': [],...",Waylon572_Feil794_4a23b7e8-7136-d3d8-3285-6dc7...


In [53]:
# def encounter_patients(filename: str):
#         df_patient = pd.read_json(filename)
#         for i in df_patient['entry'].values:
#             if i["resource"]["resourceType"]=='Condition': # and i['fullUrl'] == 'urn:uuid:9fdd4884-02d6-051a-be80-69a99ec7bd25': 
#                 print(i)
                # print(i["resource"]["code"]["text"])    
# encounter_patients(filename_test)

In [211]:
# function to check universe of results for variables based on functions defined
# e.g. info_patients -> list of resourceType, condition_patients -> conditions found in the data
# import os
# def outputs_detail(function_):
#     resources_types=[]
#     directory = "/home/cleysonl/fsdl-project/Clinical-Notes-and-DNA/fhir/"
#     for filename in os.listdir(directory):
#         f=os.path.join(directory, filename)
#         function_(f)

# outputs_detail(diagnostic_patients)    

In [78]:
# Function to see which resources apply for each patient and be able to create a list of the resources that apply
# for all patients in the clinical notes
def info_patients(filename: str):
        df_patient = pd.read_json(filename)
        l= {}
        for i in df_patient['entry'].values:
                l[i["resource"]["resourceType"]] = l.get(i["resource"]["resourceType"],0) +1
        return l

info_patients(filename_test)

{'Patient': 1,
 'Encounter': 132,
 'Condition': 10,
 'MedicationRequest': 132,
 'Claim': 264,
 'CareTeam': 5,
 'CarePlan': 5,
 'DiagnosticReport': 157,
 'DocumentReference': 132,
 'ExplanationOfBenefit': 132,
 'Observation': 339,
 'Procedure': 6,
 'Immunization': 11,
 'Provenance': 1}

In [77]:
# # Function to see which conditions apply for each patient and be able to create a list of the conditions that apply
# # for all patients in the clinical notes
def condition_patients(filename: str):
    df_patient = pd.read_json(filename)                
    l= {}
    for i in df_patient['entry'].values:
        if i["resource"]["resourceType"]=='Condition': #'ImagingStudy' and i["resource"]["series"][0]['modality']['code']=='MR':
            print(i['resource']['encounter']['reference'], i['resource']['code']['coding'][0]['display'])
            # l[i["resource"]["code"]['text']] = l.get(i["resource"]["code"]['text'],0) +1

    # return l
    
condition_patients(filename_test)

urn:uuid:ef4a387b-8d71-db15-a46e-fe68fef7bd05 Hypertension
urn:uuid:3886bbf9-a1c3-d7f5-0f44-7564a5107c33 Body mass index 30+ - obesity (finding)
urn:uuid:b19ae7e8-1014-0cce-4b94-ab17b3900585 Hyperlipidemia
urn:uuid:767bac67-a99a-6d5f-555c-bb7174d88db6 Miscarriage in first trimester
urn:uuid:81e4cf21-429c-1e2e-eacd-94961fcc2f59 Osteoporosis (disorder)
urn:uuid:9df5da12-2d76-50e3-bf01-e9c0ef0a9b97 First degree burn
urn:uuid:2a49a2f8-c8ab-8927-9016-450cd5fd619e Whiplash injury to neck
urn:uuid:ff04ca4a-fee0-6059-03fe-cb9918fefb8a Acute viral pharyngitis (disorder)
urn:uuid:34fa8111-6b75-0c90-ecb5-ef460632474b Stroke
urn:uuid:c070a983-a270-e4fd-8994-4abff2fe875b Sprain of ankle


In [68]:
def imagingstudy_patients(filename: str):
    df_patient = pd.read_json(filename)
    dict_name = df_patient.loc[0,"entry"]["resource"] ["name"][0]
    name_patient = dict_name['given'][0] + '_' + dict_name['family']
    patient_id = df_patient.loc[0,"entry"]['resource']['id']    
    n=0            
    for i in df_patient['entry'].values:
        if i["resource"]["resourceType"]=='DiagnosticReport' and i["resource"]["code"]["coding"][0]['display'].lower()=='genetic analysis summary panel':
            print(df_patient.loc[0,'entry']['resource']['name'][0])
            n+=1

    return n
    
imagingstudy_patients(filename_test)

0

In [69]:
import os
def outputs_detail(function_):
    resources_types=[]
    directory = "/home/cleysonl/fsdl-project/Clinical-Notes-and-DNA/fhir/"
    for filename in os.listdir(directory):
        f=os.path.join(directory, filename)
        if os.path.isfile(f):
            function_(f)

# outputs_detail(dnastudy_patients)    

In [70]:
# function to check universe of results for variables based on functions defined
# e.g. info_patients -> list of resourceType, condition_patients -> conditions found in the data
import os
def outputs_detail(function_):
    resources_types=[]
    directory = "/home/cleysonl/fsdl-project/Clinical-Notes-and-DNA/fhir/"
    n_total=0
    for filename in os.listdir(directory):
        f=os.path.join(directory, filename)
        if os.path.isfile(f):
            n_total+=function_(f)
    return n_total

outputs_detail(imagingstudy_patients)    

{'use': 'official', 'family': 'Rowe323', 'given': ['Kala987'], 'prefix': ['Mrs.']}
{'use': 'official', 'family': 'Cassin499', 'given': ['Inez706'], 'prefix': ['Mrs.']}
{'use': 'official', 'family': 'Satterfield305', 'given': ['Val761'], 'prefix': ['Mr.']}
{'use': 'official', 'family': 'Anderson154', 'given': ['Glinda951'], 'prefix': ['Ms.']}
{'use': 'official', 'family': 'Hilpert278', 'given': ['Andy346'], 'prefix': ['Mr.']}
{'use': 'official', 'family': 'Wolff180', 'given': ['Donovan745'], 'prefix': ['Mr.']}
{'use': 'official', 'family': 'Douglas31', 'given': ['Chong355'], 'prefix': ['Mr.']}
{'use': 'official', 'family': 'McLaughlin530', 'given': ['Ricky354'], 'prefix': ['Mr.']}
{'use': 'official', 'family': 'West559', 'given': ['Alan320'], 'prefix': ['Mr.']}
{'use': 'official', 'family': 'Feil794', 'given': ['Waylon572'], 'prefix': ['Mr.']}
{'use': 'official', 'family': 'Padberg411', 'given': ['Jimmy858'], 'prefix': ['Mr.']}
{'use': 'official', 'family': 'Cummings51', 'given': ['Tani

419

In [118]:
dict_name = df_patient.loc[0,"entry"]["resource"] ["name"][0]
dict_name['prefix'][0] + ' ' + dict_name['given'][0] + ' ' + dict_name['family']


'Mrs. Ryan260 Quitzon246'

In [119]:
df_patient.loc[0,"entry"]['resource']['id']


'41ce1a88-6529-3f5f-1576-1f1d7a8623c0'